<a href="https://colab.research.google.com/github/ivanmladek/Sentinel-Intelligence-Codex/blob/main/process_refactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title Setup and Authentication
!apt-get install -y poppler-utils tesseract-ocr libmagic-dev

# Downgrade numpy to a version compatible with other libraries
!pip install numpy==1.26.4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
libmagic-dev is already the newest version (1:5.41-3ubuntu0.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires pyarrow>=15.0.2, but you have pyarrow 14.0.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 

In [1]:
import numpy
print(numpy.__version__)

1.26.4


In [3]:


# Install nougat and its dependencies after runtime restart
!pip install transformers==4.38.2 pyarrow==14.0.1 timm==0.5.4 requests==2.31.0 albumentations==1.0.0 git+https://github.com/facebookresearch/nougat

# Install necessary libraries
!pip install   textblob langdetect
#!pip install nltk>3.8.1
import os
import logging
import pandas as pd
from transformers import AutoTokenizer
from google.colab import drive
import re
import shutil
from tqdm import tqdm
import json

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)



  Cloning https://github.com/facebookresearch/nougat to /tmp/pip-req-build-onur_75q
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/nougat /tmp/pip-req-build-onur_75q
  Resolved https://github.com/facebookresearch/nougat to commit 5a92920d342fb6acf05fc9b594ccb4053dbe8e7a
  Preparing metadata (setup.py) ... done


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os

def find_pdfs(directory):
    pdf_files = []
    for root, dirs, files in os.walk(directory, followlinks=True):
        for file in files:
            if file.endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

pdf_directory = '/content/drive/MyDrive/world_tracker/Biology'  # Replace with your directory path
pdf_files = find_pdfs(pdf_directory)
for p in pdf_files:
    print(p)

Mounted at /content/drive
/content/drive/MyDrive/world_tracker/Biology/Adamson - Food in Medieval Times (Greenwood, 2004).pdf
/content/drive/MyDrive/world_tracker/Biology/A New Ecology - Systems Perspective - S. Jorgensen, B. Fath (Elsevier, 2007) WW.pdf
/content/drive/MyDrive/world_tracker/Biology/Agutter - About Life, Concepts in Modern Biology.pdf
/content/drive/MyDrive/world_tracker/Biology/Alcohol - A Social and Cultural History (Berg, 2006).pdf
/content/drive/MyDrive/world_tracker/Biology/An Introduction to Forensic Genetics - W. Goodwin, et al., (Wiley, 2007) WW.pdf
/content/drive/MyDrive/world_tracker/Biology/Aging of the Genome - The Dual Role of DNA in Life and Death - J. Vijg (Oxford, 2007) WW.pdf
/content/drive/MyDrive/world_tracker/Biology/Atlas of Skeletal Muscles 3rd ed - R. Stone, J. Stone WW.pdf
/content/drive/MyDrive/world_tracker/Biology/Bell,Hemsley - Green Plants - Their Origin and Diversity 2e.pdf
/content/drive/MyDrive/world_tracker/Biology/Barnes - Herbal Medici

In [5]:

from concurrent.futures import ThreadPoolExecutor, as_completed
from langdetect import detect, LangDetectException

import fcntl  # Import the fcntl module for file locking
def sanitize_filename(filename):
    # Remove spaces and other potentially problematic characters
    sanitized = re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)
    return sanitized



def upload_to_drive(csv_path, drive_directory):
    try:
        #drive.mount('/content/drive', force_remount=True)
        destination_path = os.path.join(drive_directory, os.path.basename(csv_path))
        shutil.copy(csv_path, destination_path)
        print(f"Uploaded {csv_path} to {destination_path}")
    except Exception as e:
        logger.error(f"Error uploading to Google Drive: {e}")

import nltk
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.corpus import words, brown
from transformers import pipeline

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('brown')
GARBAGE_THRESHOLD = 0.8

# Load a pre-trained language model for fluency scoring
language_model = pipeline("text-generation", model="gpt2")
from textblob import TextBlob

def text_quality_score(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Check if tokens are common English words
    english_words = set(words.words())
    common_word_ratio = sum(1 for word in tokens if word.lower() in english_words) / len(tokens) if tokens else 0

    # Part-of-Speech tagging
    pos_tags = pos_tag(tokens)
    pos_score = sum(1 for word, tag in pos_tags if tag in ['NN', 'VB', 'JJ', 'RB']) / len(pos_tags) if pos_tags else 0

    # Language Model fluency score (simplified)
    lm_score = len(text.split()) / 10  # Simplified scoring

    # Spell check
    blob = TextBlob(text)
    misspelled_words = [word for word, pos in blob.tags if pos != 'NNP' and pos != 'NNPS' and word.lower() not in english_words and not word.isalpha()]
    misspelled_ratio = len(misspelled_words) / len(tokens) if tokens else 0

    # Check for too many numbers
    number_ratio = sum(1 for token in tokens if token.isdigit()) / len(tokens) if tokens else 0

    # Check for words with spaces between letters
    spaced_word_ratio = sum(1 for token in tokens if re.search(r'(\w) (\w)', token)) / len(tokens) if tokens else 0

    # Combine scores
    combined_score = (common_word_ratio + pos_score + lm_score) / 3

    # Include misspelled ratio, number ratio, and spaced word ratio in the score
    combined_score *= (1 - misspelled_ratio)
    combined_score *= (1 - number_ratio)
    combined_score *= (1 - spaced_word_ratio)

    return combined_score

def is_garbage(text, threshold=GARBAGE_THRESHOLD, LENWORD=50):
    # Calculate the quality score
    score = text_quality_score(text) #this check is now ooutsourced to the nougat OCR

    # Determine if the text is garbage based on the threshold
    if score < threshold:
        return True

    # Check for jammed words
    words = re.findall(r'\b\w+\b', text)
    for word in words:
        if len(word) > LENWORD and '_' not in word:  # heuristic to detect jammed words
            return True

    # Check for a high proportion of single-letter words
    single_letter_words = [word for word in words if len(word) == 1]
    if len(single_letter_words) > 0.1 * len(words):  # threshold for single-letter words
        return True

    #TODO check for repetition
      # Check for repetitive patterns
     # Check for repetitive loops
    loops = re.findall(r'(.+?)\1{2,}', text) #finds almost all loops but some sneak in
    #loops = re.findall(r'(.+?)\1', text) #finds almost no loops
    if loops:
        return True

    # Check if the text is predominantly in English
    try:
        if detect(text) != 'en':
            return True
    except LangDetectException:
        # If language detection fails, consider the text as garbage
        return True

    return False

import re

import re

def clean_text(text):
    # Remove extra newlines and spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r' +', ' ', text)

    # Remove leading and trailing spaces
    text = text.strip()

    # Remove any text between square brackets
    text = re.sub(r'\[[^\]]*\]', '', text)

    # Remove academic references, bracketed references, and references to tables and figures
    text = re.sub(r'\(\d+\)', '', text, flags=re.IGNORECASE)  # Remove (82)
    text = re.sub(r'\[[A-Za-z0-9]+\]', '', text, flags=re.IGNORECASE)  # Remove [RD47] and similar
    text = re.sub(r'\([\w\s]+et\s+al\., \d{4}\)', '', text, flags=re.IGNORECASE)  # Remove (Krizhevsky et al., 2014)
    text = re.sub(r'\(\w+\s+and\s+\w+\s+\d{4}\)', '', text, flags=re.IGNORECASE)  # Remove (Menendez and Woodworth 2010)
    text = re.sub(r'\(see\s+equations\s+\(\d+\)\s+and\s+\(\d+\)\)', '', text, flags=re.IGNORECASE)  # Remove (see equations (5) and (10))
    text = re.sub(r'\(\w+\s+et\s+al\., \d{4};\s*\w+\s+et\s+al\., \d{4}\)', '', text, flags=re.IGNORECASE)  # Remove (Yao et al., 2007; Lin et al., 2016)
    text = re.sub(r'Table\s+\d+', '', text, flags=re.IGNORECASE)  # Remove Table 7
    text = re.sub(r'\[FIGURE:[^]]+\]', '', text, flags=re.IGNORECASE)  # Remove [FIGURE:S4.F1]
    text = re.sub(r'\[\d+(,\s*\d+)*\]', '', text, flags=re.IGNORECASE)  # Remove [23, 32, 36, 37, 38, 39]
    text = re.sub(r'\[.*arxiv.*\]', '', text, flags=re.IGNORECASE)  # Remove any text containing "arxiv" between brackets

    # Remove any remaining non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text, flags=re.IGNORECASE)

    # Remove any sequences of punctuation or symbols that are likely to be garbled text
    text = re.sub(r'[\.,;:!?]{2,}', '', text, flags=re.IGNORECASE)

    return text

def download_pdf(file_id, output_name):
    try:
        # Assuming file_id is the source path and output_name is the destination path
        if not os.path.exists(output_name):
            shutil.copy(file_id, output_name)
            print(f"Copied {file_id} to {output_name}")
        else:
            print(f"{output_name} already exists. Skipping download.")
    except Exception as e:
        logger.error(f"Error copying {file_id} to {output_name}: {e}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret 

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:

import nltk
from nltk.tokenize import sent_tokenize


from nltk.tokenize import sent_tokenize
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
#import PyPDF2

import os
import json
from tqdm import tqdm



import fcntl


def chunk_text(content, max_size=8192):
    segments = []
    current_segment = ""
    lines = content.split('\n')

    for line in lines:
        if line.startswith(("#", "##", "###")):
            if current_segment:
                segments.extend(split_segment(current_segment.strip(), max_size))
            current_segment = ""
        else:
            current_segment += line + " "

    if current_segment:
        segments.extend(split_segment(current_segment.strip(), max_size))

    return segments

import nltk

def split_segment(segment, max_size):
    sentences = nltk.sent_tokenize(segment)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def clean_text_from_mmd(file_path, garbage_threshold=0.8):
    raw_text_path = file_path  # Assuming the file is the raw text file
    garbage_text_path = f"{os.path.splitext(file_path)[0]}_garbage_text.jsonl"
    cleaned_text_path = f"{os.path.splitext(file_path)[0]}_cleaned_text.jsonl"

    if not os.path.exists(raw_text_path):
        print(f"Raw text for {file_path} does not exist. Skipping cleaning.")
        return

    # Delete existing JSONL files
    if os.path.exists(garbage_text_path):
        os.remove(garbage_text_path)
    if os.path.exists(cleaned_text_path):
        os.remove(cleaned_text_path)

    print(f"Starting text cleaning from {file_path}")
    garbage_count = 0
    total_segments = 0

    with open(raw_text_path, "r", encoding='utf-8') as raw_f, \
         open(cleaned_text_path, "a", encoding='utf-8') as cleaned_f, \
         open(garbage_text_path, "a", encoding='utf-8') as garbage_f:
        progress_bar = tqdm(desc=f"Cleaning {file_path}", unit="segment", leave=True)

        # Read the entire file content
        content = raw_f.read()

        # Call a separate function for chunking the text
        segments = chunk_text(content)

        for segment in segments:
            #print(segment)
            try:
                cleaned, is_garb = process_segment(segment, garbage_threshold)
                if is_garb:
                    fcntl.flock(garbage_f, fcntl.LOCK_EX)
                    garbage_f.write(json.dumps({"text": cleaned}) + '\n')
                    fcntl.flock(garbage_f, fcntl.LOCK_UN)
                    garbage_count += 1
                else:
                    fcntl.flock(cleaned_f, fcntl.LOCK_EX)
                    cleaned_f.write(json.dumps({"text": cleaned}) + '\n')
                    fcntl.flock(cleaned_f, fcntl.LOCK_UN)
                total_segments += 1
            except Exception as e:
                print(f"Error processing segment: {e}")
            progress_bar.update(1)
        progress_bar.close()

    garbage_percentage = (garbage_count / total_segments) * 100 if total_segments else 0
    print(f"Completed text cleaning from {file_path}. Garbage segments: {garbage_percentage:.2f}%")



In [7]:
def extract_page_text(page):
    return page.extract_text()

def process_segment(line, garbage_threshold):
    segment = line#json.loads(line)["text"]
    cleaned = clean_text(segment)
    is_garb = is_garbage(cleaned, threshold=garbage_threshold)
    return cleaned, is_garb


def check_file_in_drive(file_path, drive_directory):
    """Check if a file exists in the specified Google Drive directory."""
    # Construct the full path in the mounted drive
    full_path = os.path.join(drive_directory, os.path.basename(file_path))

    # Check if the file exists
    return os.path.exists(full_path)

import os
import subprocess
import logging

logger = logging.getLogger(__name__)

def process_pdf(file_id, drive_directory):
    try:
        logger.info(f"Starting processing for {file_id}")

        base_filename = os.path.basename(file_id)
        sanitized_base_filename = sanitize_filename(base_filename)
        sanitized_output_name = f"{sanitized_base_filename}.pdf"
        logger.info(f"Sanitized output filename: {sanitized_output_name}")

        if not os.path.exists(sanitized_output_name):
            download_pdf(file_id, sanitized_output_name)
            logger.info(f"Downloaded {sanitized_output_name}")
        else:
            logger.info(f"{sanitized_output_name} already exists. Skipping download.")

        raw_text_path = f"{os.path.splitext(sanitized_output_name)[0]}.mmd"

        # Check if the files already exist in the drive directory
        if check_file_in_drive(raw_text_path, drive_directory):
            logger.info(f"{raw_text_path} already exists in {drive_directory}. Skipping processing.Going to cleaning")
            clean_text_from_mmd(drive_directory+"/"+raw_text_path, GARBAGE_THRESHOLD)
            return

        # Check for the presence of raw_text_path (MML file)
        if not os.path.exists(raw_text_path):
            # Use nougat for text extraction if MML file is not present
            nougat_output_path = f"{os.path.splitext(sanitized_output_name)[0]}.mmd"
            nougat_command = f"nougat {sanitized_output_name} -o {nougat_output_path} --no-skipping -m 0.1.0-base --recompute"
            !nougat {sanitized_output_name} -o ./ --no-skipping --recompute
            #result = subprocess.run(nougat_command, shell=True, check=True, capture_output=True, text=True)
            #logger.info(f"Nougat output: {result.stdout}")

            # Check for the presence of MML output from nougat
            if not os.path.exists(raw_text_path):
                logger.error(f"MML output not found for {sanitized_output_name}")
                return

        # Upload results to drive
        if os.path.exists(raw_text_path):
            upload_to_drive(raw_text_path, drive_directory)
            logger.info(f"Uploaded {raw_text_path} to {drive_directory}")

        os.remove(sanitized_output_name)  # Clean up the downloaded PDF file
        logger.info(f"Removed {sanitized_output_name}")
    except subprocess.CalledProcessError as e:
        logger.error(f"Error processing {file_id}: {e}")
        logger.error(f"Nougat stderr: {e.stderr}")
    except Exception as e:
        logger.error(f"Error processing {file_id}: {e}")


def process_pdfs(pdf_files, drive_directory):
    total_pdfs = len(pdf_files)
    progress_bar = tqdm(total=total_pdfs, desc="Processing PDFs", unit="PDF", leave=True)
    for pdf_file in pdf_files:
        print(f"Copying {pdf_file} to working directory")
        # Assuming you have a function to copy the file to the working directory
        # copy_to_working_directory(pdf_file)
        process_pdf(pdf_file, drive_directory)
        progress_bar.update(1)
    progress_bar.close()
    print("Completed processing all PDFs")

In [8]:

!nougat {sanitized_output_name} -o ./ --no-skipping --recompute

downloading nougat checkpoint version 0.1.0-small to path /root/.cache/torch/hub/nougat-0.1.0-small
config.json: 100% 557/557 [00:00<00:00, 3.21Mb/s]
pytorch_model.bin: 100% 956M/956M [00:14<00:00, 71.5Mb/s]
special_tokens_map.json: 100% 96.0/96.0 [00:00<00:00, 590kb/s]
tokenizer.json: 100% 2.04M/2.04M [00:00<00:00, 14.0Mb/s]
tokenizer_config.json: 100% 106/106 [00:00<00:00, 267kb/s]
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:

drive_directory = '/content/drive/MyDrive/'

process_pdfs(pdf_files, drive_directory)

Processing PDFs:   0%|          | 0/76 [00:00<?, ?PDF/s]

Copying /content/drive/MyDrive/world_tracker/Biology/Adamson - Food in Medieval Times (Greenwood, 2004).pdf to working directory
Copied /content/drive/MyDrive/world_tracker/Biology/Adamson - Food in Medieval Times (Greenwood, 2004).pdf to Adamson_-_Food_in_Medieval_Times__Greenwood__2004_.pdf.pdf
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0% 0/71 [00:00<?, ?it/s]INFO:root:Processing file Adamson_-_Food_in_Medieval_Times__Greenwood__2004_.pdf.pdf with 284 pages
 82% 58/71 [14:23<04:57, 22.87s/it]